In [76]:
import os
import shutil
from tqdm import tqdm
from random import shuffle
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from glob import glob

import torch

from sklearn.model_selection import train_test_split

# prepare data

In [77]:
TARGETS = {'Tiger': 0, 'Leopard': 1}

In [78]:
def get_width(path):
    image = cv2.imread(path)
    return image.shape[1]

def get_height(path):
    image = cv2.imread(path)
    return image.shape[0]

## train

In [136]:
tiger_train = pd.read_csv('data/Train_3500/Tiger/objects.csv')
leo_train = pd.read_csv('data/Train_3500/Leopard/objects.csv')

tiger_train['path'] = tiger_train.id.apply(lambda x: 'data/Train_3500/Tiger/'+x)
leo_train['path'] = leo_train.id.apply(lambda x: 'data/Train_3500/Leopard/'+x)

# –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

widths = []
heights = []

for path in tqdm(tiger_train.path):
    widths.append(get_width(path))
    heights.append(get_height(path))
    
tiger_train['width'] = widths
tiger_train['height'] = heights

# –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

widths = []
heights = []

for path in tqdm(leo_train.path):
    widths.append(get_width(path))
    heights.append(get_height(path))
    
leo_train['width'] = widths
leo_train['height'] = heights

100%|██████████| 3517/3517 [03:17<00:00, 17.82it/s]


In [137]:
# bbox - xmin, ymin, xmax, ymax
# yolo - x_c, y_c, width, height

# center_x = ( (xmax + xmin) / 2) / width
# center_y = ( (ymax + ymin) / 2) / height
# width = (xmax - xmin) / width
# height = (ymax - ymin) / height

In [144]:
tiger_train['x_c'] = tiger_train.bbox.apply(lambda x: ( (float(x.split(' ')[2]) + float(x.split(' ')[0])) / 2 ) )
tiger_train['x_c'] = (tiger_train['x_c'] / tiger_train['width']).apply(lambda x: max(0, min(1, x)))

tiger_train['y_c'] = tiger_train.bbox.apply(lambda x: ( (float(x.split(' ')[3]) + float(x.split(' ')[1])) / 2 ) )
tiger_train['y_c'] = (tiger_train['y_c'] / tiger_train['height']).apply(lambda x: max(0, min(1, x)))

tiger_train['width_box'] = tiger_train.bbox.apply( lambda x: (float(x.split(' ')[2]) - float(x.split(' ')[0])) )
tiger_train['width_box'] = (tiger_train['width_box'] / tiger_train['width']).apply(lambda x: max(0, min(1, x)))

tiger_train['height_box'] = tiger_train.bbox.apply(lambda x: (float(x.split(' ')[3]) - float(x.split(' ')[1])) )
tiger_train['height_box'] = (tiger_train['height_box'] / tiger_train['height']).apply(lambda x: max(0, min(1, x)))

tiger_train['yolo'] = tiger_train.apply(lambda x: '{} '.format(TARGETS['Tiger']) + 
                                        ' '.join([str(round(s, 4)) for s in x[-4:]]), axis=1)

# –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

leo_train['x_c'] = leo_train.bbox.apply(lambda x: ( (float(x.split(' ')[2]) + float(x.split(' ')[0])) / 2 ) )
leo_train['x_c'] = (leo_train['x_c'] / leo_train['width']).apply(lambda x: max(0, min(1, x)))

leo_train['y_c'] = leo_train.bbox.apply(lambda x: ( (float(x.split(' ')[3]) + float(x.split(' ')[1])) / 2 ) )
leo_train['y_c'] = (leo_train['y_c'] / leo_train['height']).apply(lambda x: max(0, min(1, x)))

leo_train['width_box'] = leo_train.bbox.apply( lambda x: (float(x.split(' ')[2]) - float(x.split(' ')[0])) )
leo_train['width_box'] = (leo_train['width_box'] / leo_train['width']).apply(lambda x: max(0, min(1, x)))

leo_train['height_box'] = leo_train.bbox.apply(lambda x: (float(x.split(' ')[3]) - float(x.split(' ')[1])) )
leo_train['height_box'] = (leo_train['height_box'] / leo_train['height']).apply(lambda x: max(0, min(1, x)))

leo_train['yolo'] = leo_train.apply(lambda x: '{} '.format(TARGETS['Leopard']) + 
                                    ' '.join([str(round(s, 4)) for s in x[-4:]]), axis=1)

In [145]:
train_other = pd.DataFrame()

train_other['path'] = glob('data/Train_3500/Other/*.jpg')
train_other['id'] = train_other['path'].apply(lambda x: x.split('/')[-1])
train_other['yolo'] = ''
train_other['class'] = 'Other'

In [146]:
train = pd.concat([tiger_train[['path', 'id', 'class', 'yolo']], 
                   leo_train[['path', 'id', 'class', 'yolo']], 
                   train_other[['path', 'id', 'class', 'yolo']]], axis=0)

In [147]:
train.groupby('class').path.count()

class
Leopard    3517
Other      3500
Tiger      3537
Name: path, dtype: int64

In [155]:
tiger_train[['x_c', 'y_c', 'width', 'height']].min()

x_c          0.028877
y_c          0.245768
width     1600.000000
height    1080.000000
dtype: float64

In [156]:
train.to_csv('train.csv', index=False)

## save to yolo format

In [158]:
train = pd.read_csv('train.csv')
train.yolo = train.yolo.fillna('')

In [159]:
train.groupby(['id'])['class'].nunique().sort_values()

id
001_1.jpg                        1
214_IMG_0623_S02.jpg             1
214_IMG_0624_S02.jpg             1
214_IMG_0638_S02.jpg             1
214_IMG_0640_S02.jpg             1
                                ..
003_92.jpg                       1
003_93.jpg                       1
003_96.jpg                       1
003_783.jpg                      1
добавочная_5_IMAG0044_S02.jpg    1
Name: class, Length: 10454, dtype: int64

In [161]:
for dir_ in ['data_yolo/train/images', 'data_yolo/train/labels',
             'data_yolo/val/images', 'data_yolo/val/labels']:

    shutil.rmtree(dir_)
    os.mkdir(dir_)

In [162]:
train_ann, val_ann = train_test_split(train, test_size=0.15, 
                                      stratify=train['class'], random_state=17)

In [163]:
train_ann

,path,id,class,yolo
5813,data/Train_3500/Leopard/02270351_S02.jpg,02270351_S02.jpg,Leopard,1 0.6624 0.6697 0.4062 0.3002
3090,data/Train_3500/Tiger/001_51.jpg,001_51.jpg,Tiger,0 0.3481 0.5742 0.6963 0.5052
1773,data/Train_3500/Tiger/001_1813.jpg,001_1813.jpg,Tiger,0 0.7029 0.8441 0.5415 0.2676
8714,data/Train_3500/Other/258_PICT1244_S17.jpg,258_PICT1244_S17.jpg,Other,
5448,data/Train_3500/Leopard/109_IMAG0095_S02.jpg,109_IMAG0095_S02.jpg,Leopard,1 0.5949 0.5849 0.2758 0.1802
...,...,...,...,...
8163,data/Train_3500/Other/12967.jpg,12967.jpg,Other,
46,data/Train_3500/Tiger/002_581.jpg,002_581.jpg,Tiger,0 0.5421 0.5014 0.4488 0.5535
2091,data/Train_3500/Tiger/001_2170.jpg,001_2170.jpg,Tiger,0 0.5876 0.4668 0.1937 0.7269
656,data/Train_3500/Tiger/002_1478.jpg,002_1478.jpg,Tiger,0 0.47 0.4352 0.9334 0.8704


In [164]:
for image_file in tqdm(train_ann.id.unique()):
    
    temp_ann = train_ann[train_ann.id == image_file]
    class_temp = temp_ann['class'].iloc[0]
    image = cv2.imread(temp_ann.path.iloc[0])
    
    s_ann = ''
    
    for i, val in temp_ann.iterrows():
        
        s_ann += val['yolo']
        if i != temp_ann.shape[0] - 1:
            s_ann += '\n'
    
    label_file = 'data_yolo/train/labels/'+class_temp+'_'+image_file[:-4]+'.txt'
    image_file = 'data_yolo/train/images/'+class_temp+'_'+image_file
    
    with open(label_file, 'w') as f:
        f.write(s_ann)
    
    cv2.imwrite(image_file, image)
    
    
    
# ––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––    
    
for image_file in tqdm(val_ann.id.unique()):
    
    temp_ann = val_ann[val_ann.id == image_file]
    class_temp = temp_ann['class'].iloc[0]
    image = cv2.imread(temp_ann.path.iloc[0])
    
    s_ann = ''
    
    for i, val in temp_ann.iterrows():
        
        s_ann += val['yolo']
        if i != temp_ann.shape[0] - 1:
            s_ann += '\n'
    
    label_file = 'data_yolo/val/labels/'+class_temp+'_'+image_file[:-4]+'.txt'
    image_file = 'data_yolo/val/images/'+class_temp+'_'+image_file
    
    with open(label_file, 'w') as f:
        f.write(s_ann)
    
    cv2.imwrite(image_file, image)

100%|██████████| 1582/1582 [01:38<00:00, 16.04it/s]


In [165]:
train_ann

,path,id,class,yolo
5813,data/Train_3500/Leopard/02270351_S02.jpg,02270351_S02.jpg,Leopard,1 0.6624 0.6697 0.4062 0.3002
3090,data/Train_3500/Tiger/001_51.jpg,001_51.jpg,Tiger,0 0.3481 0.5742 0.6963 0.5052
1773,data/Train_3500/Tiger/001_1813.jpg,001_1813.jpg,Tiger,0 0.7029 0.8441 0.5415 0.2676
8714,data/Train_3500/Other/258_PICT1244_S17.jpg,258_PICT1244_S17.jpg,Other,
5448,data/Train_3500/Leopard/109_IMAG0095_S02.jpg,109_IMAG0095_S02.jpg,Leopard,1 0.5949 0.5849 0.2758 0.1802
...,...,...,...,...
8163,data/Train_3500/Other/12967.jpg,12967.jpg,Other,
46,data/Train_3500/Tiger/002_581.jpg,002_581.jpg,Tiger,0 0.5421 0.5014 0.4488 0.5535
2091,data/Train_3500/Tiger/001_2170.jpg,001_2170.jpg,Tiger,0 0.5876 0.4668 0.1937 0.7269
656,data/Train_3500/Tiger/002_1478.jpg,002_1478.jpg,Tiger,0 0.47 0.4352 0.9334 0.8704


# fit yolo

In [129]:
# !git clone https://github.com/ultralytics/yolov5
# %cd yolov5
# !pip install -r requirements.txt

In [166]:
yaml_file = "yolov5/data/animal.yaml"
train_images_dir = os.path.join('..', 'data_yolo/train', 'images')
val_images_dir = os.path.join('..', 'data_yolo/val', 'images')

names_str = ''
for item in ['Tiger', 'Leopard']:
    names_str = names_str + ", \'%s\'"%item
names_str = "names: ["+names_str[1:]+"]"

with open(yaml_file, "w") as wobj:
    wobj.write("train: %s\n"%train_images_dir)
    wobj.write("val: %s\n"%val_images_dir)
    wobj.write("nc: %d\n"%2)
    wobj.write(names_str+"\n")
    

In [171]:
!cd yolov5 && python train.py --img 512 --batch 8 --epochs 20 --data ../yolov5/data/animal.yaml --weights ../yolov5/runs/train/exp3/weights/last.pt



train: weights=../yolov5/runs/train/exp3/weights/last.pt, cfg=, data=../yolov5/data/animal.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=20, batch_size=8, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-109-g7c6bae0 torch 1.9.0 CPU

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, sc

In [175]:
!cd yolov5 && python val.py --weights ../yolov5/runs/train/exp5/weights/best.pt --data ../yolov5/data/animal.yaml --img 512 --conf-thres 0.5



val: data=../yolov5/data/animal.yaml, weights=['../yolov5/runs/train/exp5/weights/best.pt'], batch_size=32, imgsz=512, conf_thres=0.5, iou_thres=0.6, task=val, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.0-109-g7c6bae0 torch 1.9.0 CPU

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning '../data_yolo/val/labels.cache' images and labels... 2306 found, 0
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all       2306       3309      0.963      0.955      0.966      0.806
               Tiger       2306       2781      0.964      0.923      0.949      0.709
             Leopard       2306        528      0.963      0.987      0.983      0.904
Speed: 0.3ms pre-process, 47.7ms inference, 0.1ms NMS per image at shape (32, 3, 512, 512)
